In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install numpy>=1.19.0 torch>=1.7.1 pytorch-lightning>=1.1.6 n2>=0.1.7


In [ ]:
!pip install recordlinkage -q
!pip install entity-embed -q
!pip install Unidecode -q
!pip install torch -q


In [ ]:
#import necessary packages
import recordlinkage
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [ ]:
import entity_embed
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [ ]:
import sys

sys.path.insert(0, '..')

##Loading datasets into pandas DataFrames

In [ ]:
facility_df = pd.read_csv("/content/drive/MyDrive/Record Linkage project/Hackathon/Data/synthetic_facility_v3.csv")
hdss_df = pd.read_csv("/content/drive/MyDrive/Record Linkage project/Hackathon/Data/synthetic_hdss_v3.csv")

In [ ]:
#Subset to use relevant variables only
facility_df = facility_df[['recnr', 'firstname', 'lastname', 'petname', 'dob', 'sex', 'nationalid', 'patientid', 'visitdate']]

facility_df.head()

In [ ]:
hdss_df.head()

PREPROCESSING

In [ ]:
#loading ncessary packages
import re
import numpy as np
import pprint
import pandas as pd
from datetime import datetime

In [ ]:
# Define ID variables
id_variables = ['nationalid', 'patientid', 'hdssid', 'hdsshhid']
#Define string variables
str_variables = ['firstname', 'lastname', 'petname']
#Define date variables
date_variables = ['dob', 'visitdate']

#Preprocessing step
def clean_field(field, column_name):
    # Check if the field is NaN
    if pd.isna(field):
        return field
    #convert to lower
    #field = field.lower()

    # Convert the field to string
    field = str(field)

    # Special handling for "nationalid" column
    if column_name in id_variables:
        # Remove all non-numeric characters
        cleaned_field = re.sub(r'\D', '', field)
        # Convert to integer
        cleaned_field = int(cleaned_field) if cleaned_field else None
    else:
        # Remove irrelevant separators
        cleaned_field = re.sub(r'[^\w\s-]', '', field)


    if column_name in str_variables:
        #Convert to lower
        cleaned_field = cleaned_field.lower()
        # Remove multiple spaces
        cleaned_field = re.sub(r'\s+', ' ', cleaned_field)

    if column_name in date_variables:
        # Parse date to datetime format
        try:
            cleaned_field = datetime.strptime(field, '%d/%m/%Y').date()
        except ValueError:
            cleaned_field = None


    # Strip leading and trailing spaces
    if isinstance(cleaned_field, str):
        cleaned_field = re.sub(r'\s+', ' ', cleaned_field.strip())

    return cleaned_field


# Function to clean all relevant fields in a row
def clean_row(row):
    cleaned_row = row.copy()
    for col in row.index:
        # Applying function to selected string, date, or ID variable
        if col in str_variables or col in id_variables or col in date_variables:
            cleaned_row[col] = clean_field(row[col], col)
    return cleaned_row

In [ ]:
print("For facilitity data types",facility_df.dtypes)
#Preprocess Facility dataset
facility_df = facility_df.apply(clean_row, axis=1)

facility_df.fillna('N/A', inplace=True)


# Print cleaned DataFrame
print(facility_df)


In [ ]:
print("For hdss data types", hdss_df.dtypes)
#Preprocess hdss dataset
hdss_df = hdss_df.apply(clean_row, axis=1)

hdss_df.fillna('N/A', inplace=True)

# Print cleaned DataFrame
print(hdss_df)

In [ ]:
""""
# Remove leading and trailing whitespaces from string columns
facility_df = facility_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
hdss_df = hdss_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
print(hdss_df.head())
facility_df.head() """

In [ ]:
print("For facilitity data types",facility_df.dtypes)
print("For hdss data types", hdss_df.dtypes)

In [ ]:
#Assuming that some datasets won't have unique identifiers as is the case with our two primary datasets recnr
#Add attribute id to each record using enumerate
from entity_embed.data_utils.utils import Enumerator

# Define enumerator and record_dict
id_enumerator = Enumerator()
record_dict = {}
source_field = '__source'
left_source = 'facility_df'
right_source = 'hdss_df'

# Allocate unique identifiers and track source file for facility_df
for index, row in facility_df.iterrows():
    allocated_id = id_enumerator[index]  # Allocate unique identifier using enumerate
    row[source_field] = left_source  # Track source file
    record_dict[allocated_id] = row

# Allocate unique identifiers and track source file for hdss_df
for index, row in hdss_df.iterrows():
    allocated_id = id_enumerator[index + len(facility_df)]  # Allocate unique identifier using enumerate
    row[source_field] = right_source  # Track source file
    record_dict[allocated_id] = row


In [ ]:
import pandas as pd

# Convert record_dict to DataFrame
df = pd.DataFrame.from_dict(record_dict, orient='index')

# Define file names for CSVs
facility_csv = 'facility_data.csv'
hdss_csv = 'hdss_data.csv'

# Split DataFrame based on source_field
facility_data = df[df[source_field] == left_source]
hdss_data = df[df[source_field] == right_source]

# Drop the source_field column
facility_data.drop(columns=[source_field], inplace=True)
hdss_data.drop(columns=[source_field], inplace=True)

# Save as CSV files
facility_data.to_csv(facility_csv)
hdss_data.to_csv(hdss_csv)


In [ ]:
import csv
pos_pair_set = set()

with open(f'/content/drive/MyDrive/Record Linkage project/Hackathon/Data/Gold standard.csv', newline='') as f:
    for row in csv.DictReader(f):
        id_left = id_enumerator[row['id_facility_df']]
        id_right = id_enumerator[row['id_hdss_df']]
        pos_pair_set.add((id_left, id_right))

len(pos_pair_set)

In [ ]:
from entity_embed.data_utils import utils

cluster_mapping, cluster_dict = utils.id_pairs_to_cluster_mapping_and_dict(pos_pair_set, record_dict)
len(cluster_dict)

In [ ]:
cluster_dict[1]

In [ ]:
[cluster_mapping[id_] for id_ in cluster_dict[1]]

In [ ]:
cluster_field = 'cluster'
utils.assign_clusters(record_dict, cluster_field, cluster_mapping)

#for id_ in cluster_dict[1]:
#    display(record_dict[id_])

In [ ]:
from entity_embed.data_utils.utils import cluster_dict_to_id_pairs

len(cluster_dict_to_id_pairs(cluster_dict) - pos_pair_set)

In [ ]:
from entity_embed.data_utils import utils

train_record_dict, valid_record_dict, test_record_dict = utils.split_record_dict_on_clusters(
    record_dict=record_dict,
    cluster_field=cluster_field,
    train_proportion=0.5,
    valid_proportion=0.2,
    random_seed=random_seed)

In [ ]:
len(train_record_dict), len(valid_record_dict), len(test_record_dict)

In [ ]:
field_list = (str_variables,"sex")


In [ ]:
print(field_list)


In [ ]:
"""
#Ensure unicode
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record in record_dict.values():
    for field in field_list:
        record[field] = clean_str(record[field])
        """

In [ ]:
from entity_embed.data_utils.field_config_parser import DEFAULT_ALPHABET

alphabet = DEFAULT_ALPHABET
''.join(alphabet)

In [ ]:
field_config_dict = {
    'firstname': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'lastname': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'petname': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'sex': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}


In [ ]:
#!pip install torchtext -q
from torchtext.vocab import Vocab
from entity_embed import FieldConfigDictParser

# Convert all values in record_dict to strings
for key, value in record_dict.items():
    record_dict[key] = str(value)

record_numericalizer = FieldConfigDictParser.from_dict(field_config_dict, record_list=record_dict.values())


In [ ]:
from entity_embed import LinkageDataModule

batch_size = 32
eval_batch_size = 64
datamodule = LinkageDataModule(
    train_record_dict=train_record_dict,
    valid_record_dict=valid_record_dict,
    test_record_dict=test_record_dict,
    source_field=source_field,
    left_source=left_source,
    cluster_field=cluster_field,
    record_numericalizer=record_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

In [ ]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    record_numericalizer,
    ann_k=ann_k,
    source_field=source_field,
    left_source=left_source,
)

In [ ]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_recall_at_0.3",
    tb_save_dir='tb_logs',
    tb_name='amazon-google',
)

In [ ]:
model.validate(datamodule)

In [ ]:
model.get_pool_weights()

## Testing

Again with the best validation model, check the performance on the test set:

In [ ]:
model.test(datamodule)

In [ ]:
test_left_vector_dict, test_right_vector_dict = model.predict(
    record_dict=test_record_dict,
    batch_size=eval_batch_size
)

In [ ]:
vis_sample_size = 10

In [ ]:
test_vector_dict = {**test_left_vector_dict, **test_right_vector_dict}
test_cluster_dict = utils.record_dict_to_cluster_dict(test_record_dict, cluster_field)
vis_cluster_dict = dict(sorted(test_cluster_dict.items(), key=lambda x: len(x[1]), reverse=True)[:vis_sample_size])

In [ ]:
vis_x = np.stack([test_vector_dict[id_] for cluster in vis_cluster_dict.values() for id_ in cluster])
vis_y = np.array([cluster_id for cluster_id, cluster in vis_cluster_dict.items() for __ in cluster])

In [ ]:
from sklearn.manifold import TSNE

tnse = TSNE(metric='cosine', perplexity=20, square_distances=True, random_state=random_seed)
tsne_results = tnse.fit_transform(vis_x)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

plt.figure(figsize=(16,10))
ax = sns.scatterplot(
    x=tsne_results[:,0],
    y=tsne_results[:,1],
    hue=vis_y,
    palette=sns.color_palette("hls", len(vis_cluster_dict.keys())),
    legend="full",
    alpha=0.8
)
for id_, (x, y) in zip(itertools.chain.from_iterable(vis_cluster_dict.values()), tsne_results):
    # text = id_
    text = test_record_dict[id_]['title'][:30]
    ax.text(x + 2, y + 2, text)